### general

In [1]:
import requests
from datetime import datetime, timedelta
import json
import msal
import webbrowser
from typing import List, Dict, Optional, Tuple
import os
os.chdir("/home/daniel/de_AI_Fabriek/belasting-agent-playground/")

from keys import O365_CLIENT_SECRET, O365_CLIENT_ID

# SCOPES = ["User.Read"]
SCOPES = ['Calendars.ReadWrite']
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'

# AUTHORITY = 'https://login.microsoftonline.com/' + "b4790366-ebc1-458b-8baf-4aed4d12f0fc"
AUTHORITY = 'https://login.microsoftonline.com/common'

# experimental

### general

In [2]:

def generate_access_token(app_id, scopes):
    # Save Session Token as a token file
    access_token_cache = msal.SerializableTokenCache()

    # read the token file
    if os.path.exists('msg_api_token_access.json'):
        access_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        token_detail = json.load(open('msg_api_token_access.json',))
        if not token_detail:
            os.remove('msg_api_token_access.json')
            access_token_cache = msal.SerializableTokenCache()
            
        if token_detail:
            token_detail_key = list(token_detail['RefreshToken'].keys())[0]
            modified = datetime.fromtimestamp(int(token_detail['RefreshToken'][token_detail_key]['last_modification_time']))
            end_date = modified + timedelta(days=90)
            if datetime.now() > end_date:
                os.remove('msg_api_token_access.json')
                access_token_cache = msal.SerializableTokenCache()

    # assign a SerializableTokenCache object to the client instance
    client = msal.PublicClientApplication(client_id=app_id, token_cache=access_token_cache)

    accounts = client.get_accounts()
    if accounts:
        # load the session
        token_response = client.acquire_token_silent(scopes, accounts[0])
    if not accounts or not token_response:
        # authetnicate your accoutn as usual
        flow = client.initiate_device_flow(scopes=scopes)
        print(json.dumps(flow, indent=4))
        print('user_code: ' + flow['user_code'])
        webbrowser.open('https://microsoft.com/devicelogin')
        token_response = client.acquire_token_by_device_flow(flow)

    with open('msg_api_token_access.json', 'w') as _f:
        _f.write(access_token_cache.serialize())

    return token_response

             


## Calendar

### Predecessors 

In [3]:
access_token = generate_access_token(O365_CLIENT_ID, SCOPES)
headers = {
    'Authorization': 'Bearer ' + access_token['access_token']
}

# create an event


In [4]:
def get_header():
    access_token = generate_access_token(O365_CLIENT_ID, SCOPES)
    headers = {
        'Authorization': 'Bearer ' + access_token['access_token']
    }
    return headers

In [5]:
# TODO: Variable time zones

In [6]:
def construct_event_detail(event_name, **kwargs):
    request_body = {
        'subject': event_name,
    }
    for key, value in kwargs.items():
        request_body[key] = value
    return request_body


### Create event

In [ ]:
response1_create = requests.post(
    GRAPH_API_ENDPOINT + f'/me/events',
    headers=headers,
    json=construct_event_detail("test_event2")
)

print(json.dumps(response1_create.json(), indent=4))

{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
    "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHElhA==\"",
    "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34lwAAAA==",
    "createdDateTime": "2025-02-05T14:19:23.2808529Z",
    "lastModifiedDateTime": "2025-02-05T14:19:23.3517688Z",
    "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHElhA==",
    "categories": [],
    "transactionId": null,
    "originalStartTimeZone": "UTC",
    "originalEndTimeZone": "UTC",
    "iCalUId": "040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD",
    "uid": "040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD",
    "reminderMinutesBeforeStart": 15,
    "isReminderOn": true,
    "hasAttachments": false,
    "subject":

In [30]:
print(json.dumps(response1_create.json(), indent=4))


{
  "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
  "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHEdew==\"",
  "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34jQAAAA==",
  "createdDateTime": "2025-02-05T11:38:41.7287303Z",
  "lastModifiedDateTime": "2025-02-05T11:38:41.7705395Z",
  "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHEdew==",
  "categories": [],
  "transactionId": null,
  "originalStartTimeZone": "UTC",
  "originalEndTimeZone": "UTC",
  "iCalUId": "040000008200E00074C5B7101A82E00800000000BF6E7B81C277DB010000000000000000100000003153086DC0AB3146AED38B48FB37B007",
  "uid": "040000008200E00074C5B7101A82E00800000000BF6E7B81C277DB010000000000000000100000003153086DC0AB3146AED38B48FB37B007",
  "reminderMinutesBeforeStart": 15,
  "isReminderOn": true,
  "hasAttachments": false,
  "subject": "test_event2",
  "bodyPreview":

In [ ]:
# TODO: figure out attendees

In [ ]:
# Step 2.2 Example (Detailed)
event_name = 'Work Holiday'
body = {
    # html or text
    'contentType': 'html',
    'content': '<b>2 weeks vacation</b>'
}
start = {
    'dateTime': '2025-02-20T09:00:00',
    'timeZone': 'Europe/Amsterdam'
}
end = {
    'dateTime': '2025-02-21T17:00:00',
    'timeZone': 'Europe/Amsterdam'
}
location = {
    'displayName': 'Paris, France'
}
attendees = [
    {
        'emailAddress': {
            'address': 'daniel@vanoosteroom.com'
        },
         'type': 'required' # or optional
    }
]

In [24]:
response2_create = requests.post(
    GRAPH_API_ENDPOINT + f'/me/events',
    headers=headers,
    json=construct_event_detail(
            event_name,
            body=body,
            location=location,
            start=start,
            end=end,
            #attendees=attendees,
        )
)

NameError: name 'event_name' is not defined

In [22]:
print(json.dumps(response2_create.json(), indent=4))

{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_1EBDCD179CAEDECE%40outlook.com')/events/$entity",
    "@odata.etag": "W/\"ClAZo7UKNEq0Gv3bsgM5dQAADQpt/g==\"",
    "id": "AQMkADAwATMwMAExLThjNTUALTAwAGJlLTAwAi0wMAoARgAAAxp5qsgLJ-hOuMYmhH_7aJYHAApQGaO1CjRKtBr927IDOXUAAAIBDQAAAApQGaO1CjRKtBr927IDOXUAAAANC2WsAAAA",
    "createdDateTime": "2025-02-19T10:40:25.3885764Z",
    "lastModifiedDateTime": "2025-02-19T10:40:25.4594125Z",
    "changeKey": "ClAZo7UKNEq0Gv3bsgM5dQAADQpt/g==",
    "categories": [],
    "transactionId": null,
    "originalStartTimeZone": "Europe/Amsterdam",
    "originalEndTimeZone": "Europe/Amsterdam",
    "iCalUId": "040000008200E00074C5B7101A82E00800000000F57347AFBA82DB01000000000000000010000000941F0F9A96B53F42B1DA0E773ECB8323",
    "uid": "040000008200E00074C5B7101A82E00800000000F57347AFBA82DB01000000000000000010000000941F0F9A96B53F42B1DA0E773ECB8323",
    "reminderMinutesBeforeStart": 15,
    "isReminderOn": true,
    "hasAttachmen

### Auto create work week

In [8]:

import random

week_schedule = []

for _ in range(5):

    times = [(timedelta(hours=9) + timedelta(minutes=random.choice([0, 15, 30, 45])))]


    while True:
        random_value = times[-1] + timedelta(minutes=random.choice(range(30, 180, 15)))
        times.append(random_value)
        if random_value >= timedelta(hours=16, minutes=30):
            times.pop(-1)
            times.append(timedelta(hours=17))
            break
    week_schedule.append(times)
    
# for i, day_schedule in enumerate(week_schedule):
    # print('Day')
    # for j, time in enumerate(day_schedule):
        # print(time)

In [9]:
def write_event(event_name, start, end, **kwargs):
    response = requests.post(
        GRAPH_API_ENDPOINT + f'/me/events',
        headers=headers,
        json=construct_event_detail(
            event_name,
            start=start,
            end=end,
            **kwargs
        )
    )
    return response

In [10]:
# Create a randon da

def to_iso_date(date):
    return date.strftime("%Y-%m-%d")

def to_iso_time(time):
    string_time = str(time)
    dt = datetime.strptime(string_time, "%H:%M:%S")
    return dt.strftime("%H:%M:%S")

def to_iso(date, time):
    return to_iso_date(date) + 'T' + to_iso_time(time)

def time_json(date, time):
    return {
        'dateTime': to_iso(date, time),
        'timeZone': 'Europe/Amsterdam'
    }
    
def convert_to_date_time(time):
    hours, minutes, seconds = map(int, str(time).split(':'))
    time_checked = timedelta(hours=hours, minutes=minutes, seconds=seconds)
    return time_checked



time_lenght = 60
rand_day = random.randint(0, 5)
time = random.choice(range(10 * 60, 17 * 60 - time_lenght, 15))
print(timedelta(minutes=time))
print(rand_day)

checked = 0

log = []

print(timedelta(minutes=time))  
day = datetime.now() - timedelta(days=1)
for day_schedule in week_schedule:
    if day.weekday() == 4:
        day = day + timedelta(days=2)
    elif day.weekday() == 5:
        day = day + timedelta(days=1)
    
    day = day + timedelta(days=1)
        
    for j, time_indices in enumerate(day_schedule):
        log.append(write_event(
            'Meeting',

                time_json(day, day_schedule[j -1]),
                time_json(day, time_indices)
        ))

12:45:00
2
12:45:00


In [11]:
log[0].json()

{'error': {'code': 'ErrorPropertyValidationFailure',
  'message': 'At least one property failed validation.'}}

In [12]:
for i , time_indices in enumerate(day_schedule):
    print(i)

0
1
2
3
4
5


In [13]:
print(datetime.now().weekday())

3


In [14]:
random_value 

datetime.timedelta(seconds=59400)

In [15]:
"""
    
    start = datetime.now() + timedelta(days=random.randint(1, 30))
    end = start + timedelta(hours=random.randint(1, 5))
    """

'\n    \n    start = datetime.now() + timedelta(days=random.randint(1, 30))\n    end = start + timedelta(hours=random.randint(1, 5))\n    '

### get events

In [ ]:

start_datetime = (datetime.now() - timedelta(day=1)).isoformat()
end_datetime = (datetime.now() + timedelta(days=8)).isoformat()
appointments_ids = set()

while True:
    response1_get = requests.get(
    GRAPH_API_ENDPOINT + f'/me/calendar/calendarView?startDateTime={start_datetime}&endDateTime={end_datetime}',
    headers=headers
    )
    
    for i in response1_get.json()['value']:
        appointments_ids.add(i['id'])
    
    if len(appointments_ids) == 0:
        break
        
    new_start_datetime = response1_get.json()['value'][-1]['end']['dateTime']

    if start_datetime == new_start_datetime:
        break
    else:
        start_datetime = new_start_datetime
    

In [58]:
print(len(appointments_ids))

0


### Delete event

In [22]:
for i in appointments_ids:
    response = requests.delete(
        GRAPH_API_ENDPOINT + f'/me/events/{i}',
        headers=headers
    )
    response

NameError: name 'appointments_ids' is not defined

### Get and Delete

In [23]:
def get_and_delete(start_datetime = (datetime.now() - timedelta(days=1)).isoformat(),
                    end_datetime = (datetime.now() + timedelta(days=8)).isoformat()):
    headers = get_header()
    appointments_ids = set()
    while True:
        response1_get = requests.get(
        GRAPH_API_ENDPOINT + f'/me/calendar/calendarView?startDateTime={start_datetime}&endDateTime={end_datetime}',
        headers=headers
        )
        
        for i in response1_get.json()['value']:
            appointments_ids.add(i['id'])
        
        if len(appointments_ids) == 0:
            print(len(appointments_ids))
            break
            
        new_start_datetime = response1_get.json()['value'][-1]['end']['dateTime']

        if start_datetime == new_start_datetime:
            break
        else:
            start_datetime = new_start_datetime
    
    for i in appointments_ids:
        response = requests.delete(
            GRAPH_API_ENDPOINT + f'/me/events/{i}',
            headers=headers
        )
    print(f"{len(appointments_ids)} appointments deleted")
         
get_and_delete()        

25 appointments deleted


### Find open space

In [16]:
def retrieve_unique_events(start_datetime = (datetime.now() - timedelta(days=1)).isoformat(),
                    end_datetime = (datetime.now() + timedelta(days=8)).isoformat()):
    headers = get_header()
    appointments_ids = set()
    appointments_jsons = []
    
    while True:
        response1_get = requests.get(
        GRAPH_API_ENDPOINT + f'/me/calendar/calendarView?startDateTime={start_datetime}&endDateTime={end_datetime}',
        headers=headers
        )
        
        for i in response1_get.json()['value']:
            if i['id'] not in appointments_ids:
                appointments_ids.add(i['id'])
                appointments_jsons.append(i)
        
        if len(appointments_ids) == 0:
            print(len(appointments_ids))
            break
            
        new_start_datetime = response1_get.json()['value'][-1]['end']['dateTime']

        if start_datetime == new_start_datetime:
            break
        else:
            start_datetime = new_start_datetime
    return appointments_jsons

In [17]:
unique_events = retrieve_unique_events()

In [18]:
from typing import List, Dict, Optional, Tuple
 
def convert_to_timedelta(date: datetime) -> timedelta:
    return timedelta(hours=date.hour, minutes=date.minute, seconds=date.second)

def find_first_opening(appointments: List[Dict], time_lenght: int) -> Optional[Tuple[datetime, datetime]]:
    WORK_START = datetime.strptime("09:00:00", "%H:%M:%S").time()
    WORK_END = datetime.strptime("17:00:00", "%H:%M:%S").time()
    MIN_DURATION = timedelta(minutes=time_lenght)

    appointments_dt = [
        (datetime.strptime(i['start']['dateTime'][:19], "%Y-%m-%dT%H:%M:%S") + timedelta(hours=1),
         datetime.strptime(i['end']['dateTime'][:19], "%Y-%m-%dT%H:%M:%S") + timedelta(hours=1))
        for i in appointments
    ]

    appointments_dt.sort()

    merged_appointments = []
    for start, end in appointments_dt:
        if merged_appointments and (start < merged_appointments[-1][1]):
            inserted = (min(min(merged_appointments[-1][0], start), merged_appointments[-1][0]), max(merged_appointments[-1][1], end))
            merged_appointments[-1] = inserted
        else:
            merged_appointments.append((start, end))

    # Check if there is an opening before the first appointment        
    if merged_appointments and (convert_to_timedelta(merged_appointments[0][0]) - convert_to_timedelta(WORK_START)) >= MIN_DURATION:
        print("this")
        return (merged_appointments[0][0] + convert_to_timedelta(WORK_START), merged_appointments[0][0])

    # Check if there is an opening between appointments
    for i in range(len(merged_appointments) - 1):
        end_current = merged_appointments[i][1]
        start_next = merged_appointments[i + 1][0]
        if (convert_to_timedelta(start_next) - convert_to_timedelta(end_current)) >= MIN_DURATION:
            return (end_current, end_current + MIN_DURATION)

    # Check if there is an opening after the last appointment
    if merged_appointments and (convert_to_timedelta(WORK_END) - convert_to_timedelta(merged_appointments[-1][1])) >= MIN_DURATION:
        print("that")
        return (merged_appointments[-1][1], merged_appointments + MIN_DURATION)

    return None

In [19]:
def date_time_to_iso(date: datetime) -> str:
    return date.strftime("%Y-%m-%dT%H:%M:%S")

def date_time_to_dateTimeZone(date: datetime) -> Dict:
    return {
        'dateTime': date_time_to_iso(date),
        'timeZone': 'Europe/Amsterdam'
    }
    
def create_event(event_name: str, start: datetime, end: datetime, **kwargs) -> Dict:
    response = requests.post(
        GRAPH_API_ENDPOINT + f'/me/events',
        headers=headers,
        json=construct_event_detail(
            event_name,
            start=date_time_to_dateTimeZone(start),
            end=date_time_to_dateTimeZone(end),
            **kwargs
        )
    )
    return response.json()

In [20]:
found_time = find_first_opening(unique_events, 60)
print(found_time)

(datetime.datetime(2025, 2, 21, 13, 30), datetime.datetime(2025, 2, 21, 14, 30))


In [21]:
create_event('Meeting found', found_time[0], found_time[1])

{'@odata.context': "https://graph.microsoft.com/v1.0/$metadata#users('outlook_1EBDCD179CAEDECE%40outlook.com')/events/$entity",
 '@odata.etag': 'W/"ClAZo7UKNEq0Gv3bsgM5dQAADkQp/w=="',
 'id': 'AQMkADAwATMwMAExLThjNTUALTAwAGJlLTAwAi0wMAoARgAAAxp5qsgLJ-hOuMYmhH_7aJYHAApQGaO1CjRKtBr927IDOXUAAAIBDQAAAApQGaO1CjRKtBr927IDOXUAAAAORTvQAAAA',
 'createdDateTime': '2025-02-20T12:52:57.312164Z',
 'lastModifiedDateTime': '2025-02-20T12:52:57.3721851Z',
 'changeKey': 'ClAZo7UKNEq0Gv3bsgM5dQAADkQp/w==',
 'categories': [],
 'transactionId': None,
 'originalStartTimeZone': 'Europe/Amsterdam',
 'originalEndTimeZone': 'Europe/Amsterdam',
 'iCalUId': '040000008200E00074C5B7101A82E008000000000030685D9683DB010000000000000000100000006C9D5750E50E9C43A3FF0506A7FE7371',
 'uid': '040000008200E00074C5B7101A82E008000000000030685D9683DB010000000000000000100000006C9D5750E50E9C43A3FF0506A7FE7371',
 'reminderMinutesBeforeStart': 15,
 'isReminderOn': True,
 'hasAttachments': False,
 'subject': 'Meeting found',
 'bodyPre

# Self build


In [91]:
def aquire_and_use_token(app_id, scopes, authority):
    access_token_cache = msal.SerializableTokenCache()
    
    # read token cache
    if os.path.exists("msg_api_token_access.json"):
        access_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        token_detail = json.load(open('msg_api_token_access.json',))
        token_detail_key = list(token_detail['RefreshToken'].keys())[0]
        modified = datetime.fromtimestamp(int(token_detail['RefreshToken'][token_detail_key]['last_modification_time']))
        end_date = modified + timedelta(days=90)
        if datetime.now() > end_date:
            os.remove('msg_api_token_access.json')
            access_token_cache = msal.SerializableTokenCache()

        
    client = msal.PublicClientApplication(
        app_id,
        authority=authority,
        token_cache=access_token_cache
    )
    
    token_response = None
     
    accounts = client.get_accounts()
                  
    if accounts:
        
        # Assuming the end user chose this one to proceed
        chosen_account = accounts[0]    
        
        # Now let's try to find a token in cache for this account
        # or by using refresh token
        token_response = client.acquire_token_silent(scopes, account=chosen_account)

    if not token_response:
        flow = client.initiate_device_flow(scopes=scopes)
        print('user code: ', flow['user_code'])
        webbrowser.open(flow['verification_uri'])
        
        token_response = client.acquire_token_by_device_flow(flow)
        print(flow["message"])
    
    if "access_token" in token_response:
        print("Token was obtained from:", token_response["token_source"])
        if os.getenv('ENDPOINT'):
            # Calling a web API using the access token
            api_result = requests.get(
                os.getenv('ENDPOINT'),
                hearders={'Authorization': 'Bearer ' + token_response["access_token"]},
            ).json()    
            print("Web API call result", json.dumps(api_result, indent=2))
        else:
            print("Token acquisition result", json.dumps(token_response, indent=2))
    else:
        print("Token acquisition failed", token_response)
    
    # write to persistent cache
    with open("msg_api_token_access.json", "w") as _f:
        _f.write(access_token_cache.serialize())
    return token_response


In [92]:
token_response = aquire_and_use_token(O365_CLIENT_ID, SCOPES, AUTHORITY)

Token was obtained from: cache
Token acquisition result {
  "access_token": "EwB4A8l6BAAUBKgm8k1UswUNwklmy2v7U/S+1fEAAYFjwKH/x5Ln6Fj6M2BkuT+/aZjtXFGhvuuru/H5vz+p2mCG+Bt+47XJ3+fUSeOxPD7XfIQW1VkRxCkPu+HfhHscjo7r9MzcZx9Ud33NRvH8yUkApb2+/U483idZ9XxDm4Bsi2+dADo07M7HLyMwyiFAX7YL55X1zTt8sKbGJbuDrnBdn+Swnls+6kOLY2pNf5dzjXzX+P48ehBOB/B7wSq9S1jt0VKoIu6dlfxE+6DTJo0W8K8IhLCV07IiU3N4VPgTmmIqSv2Gk3yquaA4t6L2P8RAVfES+WRSY/G0miJFf2/y63284GDmrgnNFKxhmRV5AbYWD18hwQIdm2jAdVUQZgAAEPv7GnrGhFkqfn2zKUPXn11AAo+MroxLNordIhTYogoRCg7+Q03P/cNUEKkY2RhhoPB3+rFZiGTii0ZVdFEscA0Fz34RY0lSRaqLuKUiLXkg5rwHNPZTJn/GiXixghvdS2AzihW1qSO+jQL2e/Leaqpdm1tk14q295k5W6YHaBLVFgnjQxivblh2c+a1eZlGYNiN/GiXGc6Z7T0tICaJoyhPfxTyfRhO6IUirzTBLzvC01q+UGHf6yc94+IvjS4F7YRvh2+S7cdnRaAbrtRXYdSfK+PuNDmP+amuEIRCV7elKP1NeTuVL4C6s2vQfeiIFUWQux3zbQCEUgRq2qniW/7ryeohMCdrAOSqldtXAUTpXkCM/Zdz95BCF/WcyR4pZHHxsKWKz6YKf0OsUrf4t6dxMrqIH2vP3y2MLyLqAYB6VGgXbq66BcfSW/UeA5gwihVPNK7uI5g+R6SPxM0X4JsmML9ao9+rw417zqd7/M46HvWMcnyQG1OpeZjf7KB1NXJJdCk4pbqTcFAiMpcvkad

In [5]:
token_response


{'token_type': 'Bearer',
 'scope': 'openid profile User.Read email',
 'expires_in': 4362,
 'ext_expires_in': 4362,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6ImlSR2ViNUdrV2FUS0wzbXJIM1hXSXl4clJJaHV2R3pfRkE3dWRNUXdzRzAiLCJhbGciOiJSUzI1NiIsIng1dCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyIsImtpZCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wMDMwZTJmOC05Njg4LTQ5YzgtYWU0Zi1jZWY2N2E3ZWM4ZmYvIiwiaWF0IjoxNzM4NzA4NTY4LCJuYmYiOjE3Mzg3MDg1NjgsImV4cCI6MTczODcxMzIzMSwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhaQUFBQXVvMGVPQlhCVUtYSDh1WEhQak9hMDRVK2RhZm9yQ3JyS2hEYllzbTg3K3hrNmN3Wmc3bEVJcElmekdQbWc3dmUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6ImJlbGFzdGluZy1hYW5naWZ0ZW4tYWdlbnQiLCJhcHBpZCI6IjZmZTRjOTlhLTc4ODYtNDQ2My05MWJiLWNmMTMzYzVkZDE4NCIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoidmFuIE9vc3Rlcm9vbSIsImdpdmVuX25hbWUiOiJEYW5pZWwiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIyMDAxOjFjMDA6ZDA0OjIyMDA6ZmUyMDo4ND